<h2 style="color: #004C47;">Installation des bibliothèques nécessaires </h2>
<div style="border: 2px solid #01796F; background-color: #E0F2F1; color: #01796F; padding: 15px; border-radius: 5px; font-size: 16px;">
    <strong>
    
**pip install beautifulsoup4**

**pip install requests**

**pip install PyPDF2**

**pip install pdfplumber**   </div>


In [12]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import pdfplumber
import re
import time
import urllib.parse
import io
import logging
from datetime import datetime
import PyPDF2
from io import BytesIO
import os
from threading import Timer
from concurrent.futures import ThreadPoolExecutor 

<h2 style="color: #004C47;">Chargement et rechargement du module tools</h2> 

In [ ]:
import importlib
import tools

importlib.reload(tools)

<h2 style="color: #004C47;">1. Scraper les articles bioRxiv liés à la bioinformatique</h2>


<div style="border: 2px solid #01796F; background-color: #E0F2F1; color: #01796F; padding: 15px; border-radius: 5px; font-size: 16px;">
    <strong>Objectif:</strong> Ce script permet d'archiver automatiquement les dépôts logiciels non encore archivés dans la base Software Heritage.Récupérer les articles publiés sur bioRxiv dans le domaine de la bioinformatique, en collectant les informations essentielles pour une analyse et un traitement ultérieurs.

Informations collectées :

	•Titre de l’article : Titre complet décrivant le sujet de recherche.
	•Résumé (abstract) : Vue d’ensemble concise du contenu scientifique.
	•Auteurs : Liste des auteurs, ordonnée telle qu’elle apparaît sur bioRxiv.
	•Date de publication : Date de mise en ligne pour le suivi des recherches.
	•DOI : Identifiant unique facilitant la référence et l’accès.
	•URL de l’article : Lien direct vers la page de l’article sur bioRxiv.
	•Mots-clés : Mots-clés associés pour décrire le domaine ou sujet de recherche.
	•URLs des dépôts logiciels : Liens vers les dépôts de code (GitHub, GitLab) associés aux ressources logicielles.

 
</div>

In [ ]:
import tools

def scrape_articles(total_pages=10):
    """Scrape uniquement les nouveaux articles bioRxiv et les enregistre dans la table 'articles' de la nouvelle base."""
    url = 'https://www.biorxiv.org'
    conn, c = tools.create_db("bioinformatics_article.db")  # Utilise la nouvelle base de données

    for page in range(1, total_pages + 1):
        print(f"Scraping page {page} sur {total_pages}")
        response = requests.get(f'{url}/collection/bioinformatics?page={page}')
        soup = BeautifulSoup(response.content, 'html.parser')
        articles = soup.find_all('a', class_='highwire-cite-linked-title')

        for article in articles:
            title = article.text.strip()
            link = url + article['href']

            article_response = requests.get(link)
            article_soup = BeautifulSoup(article_response.content, 'html.parser')

            doi_tag = article_soup.find('meta', {'name': 'citation_doi'})
            doi = doi_tag['content'] if doi_tag else 'DOI non disponible'

            # Vérifie si l'article est déjà dans la base de données
            c.execute("SELECT * FROM articles WHERE doi = ?", (doi,))
            if c.fetchone():
                print(f"L'article avec DOI {doi} existe déjà dans la base. Arrêt du scraping.")
                tools.close_db(conn)
                return  # Sortir de la fonction car tous les articles suivants existent déjà

            # Si le DOI est nouveau, nous récupérons les autres informations
            date_tag = article_soup.find('meta', {'name': 'citation_publication_date'})
            date = date_tag['content'] if date_tag else 'Date non disponible'
            if date != 'Date non disponible':
                date = date.replace('/', '-')
                date = datetime.strptime(date, '%Y-%m-%d').date().isoformat()

            pdf_links = article_soup.find_all('a')
            pdf_link = next((url + pdf['href'] for pdf in pdf_links if 'PDF' in pdf.text and pdf['href'].endswith('.pdf')), 'Lien PDF non disponible')

            # Extraire l'abstract et nettoyer les balises HTML
            abstract_tag = article_soup.find('meta', {'name': 'citation_abstract'})
            if abstract_tag:
                raw_abstract = abstract_tag['content']
                clean_abstract = BeautifulSoup(raw_abstract, "html.parser").get_text()  # Enlever les balises HTML
            else:
                clean_abstract = 'Abstract non disponible'

            # Insertion du nouvel article dans la base de données avec clean_abstract
            c.execute('''
                INSERT INTO articles (title, link, doi, date, pdf_link, abstract)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (title, link, doi, date, pdf_link, clean_abstract))  # Utiliser clean_abstract ici
            print(f"Article ajouté : {title}")
            conn.commit()  # Valide la transaction immédiatement

            time.sleep(1)

    # Ferme la base de données
    tools.close_db(conn)

# Exécuter la fonction de scraping
scrape_articles()

<h2 style="color: #004C47;">2. Extraction des URLs des Dépôts Logiciels</h2>


<div style="border: 2px solid #01796F; background-color: #E0F2F1; color: #01796F; padding: 15px; border-radius: 5px; font-size: 16px;">
    <strong>Objectif : </strong> Extraire automatiquement les liens vers les dépôts logiciels (GitHub, GitLab) mentionnés dans les articles bioRxiv liés à la bioinformatique. Ce script parcourt les PDF et les résumés (abstracts) des articles pour identifier et collecter ces URLs de dépôt, qui seront ensuite utilisés pour archivage ou analyse.

 </div>


In [ ]:


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def extract_repository_links(db_name="bioinformatics_article.db"):
    conn, c = tools.create_db(db_name)
    c.execute("SELECT id, title, abstract, pdf_link FROM articles WHERE is_article_processed = 0")
    articles = c.fetchall()

    def process_article(article):
        article_id, title, abstract, pdf_link = article
        logging.info(f"Traitement de l'article '{title}' (ID: {article_id})")

        # Extraction des liens de dépôt
        repo_links = set()
        if abstract:
            repo_links.update(tools.extract_repository_urls(abstract))
        if pdf_link and pdf_link != 'Lien PDF non disponible':
            repo_links.update(tools.extract_links_from_pdf(pdf_link))

        if repo_links:
            logging.info(f"{len(repo_links)} liens de dépôt trouvés pour l'article '{title}'")
            for repo_url in repo_links:
                code_repo_id = tools.insert_code_repository(conn, repo_url)
                if code_repo_id:
                    tools.link_article_to_repo(conn, article_id, code_repo_id)
            c.execute("UPDATE articles SET contains_valid_repo_link = 1 WHERE id = ?", (article_id,))
        else:
            logging.info(f"Aucun lien de dépôt valide trouvé pour l'article '{title}'")

        # Mise à jour pour indiquer que l'article a été traité
        c.execute("UPDATE articles SET is_article_processed = 1 WHERE id = ?", (article_id,))
        conn.commit()
        time.sleep(1)

    # Utiliser le multithreading pour traiter les articles en parallèle
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(process_article, articles)

    tools.close_db(conn)

# Exécuter la fonction
extract_repository_links()

<h2 style="color: #004C47;">3. Archivage via Software Heritage</h2>

<div style="border: 2px solid #01796F; background-color: #E0F2F1; color: #01796F; padding: 15px; border-radius: 5px; font-size: 16px;">
    <strong>Objectif : </strong>  Assurer l’archivage à long terme des dépôts logiciels associés aux articles via l’API de Software Heritage.

	•Vérification : Pour chaque dépôt, le script vérifie s’il est déjà archivé.
	•Soumission automatique : Si non archivé, une demande d’archivage est soumise automatiquement.
	•Mise à jour : La base de données est mise à jour avec les liens et dates d’archivage.

 </div>

In [ ]:

from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def archive_repositories(db_name="bioinformatics_article.db"):
    # Création initiale de la base de données sans l'utiliser directement dans les threads
    conn, c = tools.create_db(db_name)
    c.execute("SELECT code_repo_id, code_repo_url FROM code_repositories WHERE is_archived_in_swh = 0")
    repos = c.fetchall()
    tools.close_db(conn)  # Fermer la connexion principale ici, pour ne pas l'utiliser dans les threads

    # Passer `db_name` au lieu de `conn` à chaque thread
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(process_repo, db_name, repo) for repo in repos]
        for future in as_completed(futures):
            try:
                future.result()
            except Exception as e:
                logging.error(f"Erreur lors de l'archivage d'un dépôt : {e}")

def process_repo(db_name, repo):
    conn, c = tools.create_db(db_name)  # Nouvelle connexion pour chaque thread
    code_repo_id, code_repo_url = repo
    logging.info(f"Vérification de l'archivage pour le dépôt : {code_repo_url}")

    for attempt in range(tools.MAX_RETRIES):
        is_archived, archive_date, archive_link = tools.check_archived(code_repo_url)
        
        if is_archived is None:
            logging.error(f"Impossible de vérifier l'archive pour le dépôt {code_repo_url} (tentative {attempt + 1}/{tools.MAX_RETRIES}).")
            wait_time = tools.WAIT_INTERVAL * (attempt + 1)
            logging.info(f"Attente de {wait_time} secondes avant de réessayer...")
            time.sleep(wait_time)  # Délai progressif avant de réessayer
            continue  # Réessaie après l'attente

        if is_archived:
            logging.info(f"Dépôt déjà archivé. Mise à jour des informations pour le dépôt {code_repo_url}")
            c.execute("""
                UPDATE code_repositories
                SET is_archived_in_swh = 1, swh_archive_link = ?, swh_date_last_archive = ?
                WHERE code_repo_id = ?
            """, (archive_link, archive_date, code_repo_id))
        else:
            archived = tools.archive_repo(code_repo_url)
            if archived:
                logging.info(f"Dépôt soumis pour archivage : {code_repo_url}")
                c.execute("UPDATE code_repositories SET is_archived_in_swh = 2 WHERE code_repo_id = ?", (code_repo_id,))
            else:
                logging.error(f"Échec de la soumission pour le dépôt {code_repo_url}")

        conn.commit()
        break  # Sortir de la boucle si succès

    tools.close_db(conn)  # Fermez la connexion proprement

# Exécuter la fonction principale
archive_repositories()

<h2 style="color: #004C47;">4. Re-vérification de l’archivage pour les dépôts soumis</h2>

<div style="border: 2px solid #01796F; background-color: #E0F2F1; color: #01796F; padding: 15px; border-radius: 5px; font-size: 16px;">
    <strong>Objectif : </strong>  Vérifier l’état d’archivage des dépôts soumis pour archivage auprès de Software Heritage.

	•Vérification périodique : Ce script revisite les dépôts soumis (ceux en attente d’archivage) pour voir si l’archivage a été complété.
	•Mise à jour : Si un dépôt est archivé avec succès, le script enregistre le lien et la date d’archivage dans la base de données.
</div>


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
import tools

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def recheck_archived_repositories(db_name="bioinformatics_article.db"):
    # Création initiale de la base de données pour récupérer les dépôts sans utiliser cette connexion dans les threads
    conn, c = tools.create_db(db_name)
    c.execute("SELECT code_repo_id, code_repo_url FROM code_repositories WHERE is_archived_in_swh = 2")
    repos = c.fetchall()
    tools.close_db(conn)  # Fermer la connexion principale ici, pour ne pas l'utiliser dans les threads

    # Utilisation du ThreadPoolExecutor avec un maximum de 5 threads
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_repo = {executor.submit(process_repo, db_name, repo): repo for repo in repos}
        for future in as_completed(future_to_repo):
            repo = future_to_repo[future]
            try:
                future.result()
            except Exception as e:
                logging.error(f"Erreur de traitement dans le thread pour le dépôt {repo[1]}: {e}")

def process_repo(db_name, repo):
    # Chaque thread ouvre sa propre connexion à la base de données
    conn, c = tools.create_db(db_name)
    code_repo_id, code_repo_url = repo
    logging.info(f"Re-vérification de l'archivage pour le dépôt : {code_repo_url}")

    try:
        is_archived, archive_date, archive_link = tools.check_archived(code_repo_url)
        if is_archived:
            logging.info(f"Dépôt archivé après soumission. Mise à jour pour le dépôt {code_repo_url}")
            c.execute(
                "UPDATE code_repositories SET is_archived_in_swh = 1, swh_archive_link = ?, swh_date_last_archive = ? WHERE code_repo_id = ?",
                (archive_link, archive_date, code_repo_id)
            )
        conn.commit()
    except Exception as e:
        logging.error(f"Erreur lors de la re-vérification du dépôt {code_repo_url}: {e}")
    finally:
        # Fermez la connexion proprement
        tools.close_db(conn)

# Exécuter la fonction
recheck_archived_repositories()

<h2 style="color: #004C47;">Affichage des tables et colonnes de la base de données </h2>

<div style="border: 2px solid #01796F; background-color: #E0F2F1; color: #01796F; padding: 15px; border-radius: 5px; font-size: 16px;">
    <strong>Objectif : </strong> Visualiser la structure de la base de données en restant sur le fichier jupyter pour confirmer la présence des tables attendues et des colonnes associées.
</div>

In [ ]:
import sqlite3

def show_tables():
    """Affiche le contenu des tables articles, code_repositories et articles_code_repositories 
    dans la base de données bioinformatics_article.db avec les noms des colonnes."""
    
    # Connexion à la base de données
    conn = sqlite3.connect("bioinformatics_article.db")
    c = conn.cursor()

    # Fonction pour afficher le contenu d'une table
    def show_table_content(table_name):
        print(f"\nContenu de la table '{table_name}':")
        c.execute(f"SELECT * FROM {table_name}")
        columns = [description[0] for description in c.description]
        print(columns)  # Affiche les noms des colonnes
        rows = c.fetchall()
        for row in rows:
            print(row)

    # Afficher le contenu des tables
    show_table_content("articles")
    show_table_content("code_repositories")
    show_table_content("articles_code_repositories")

    # Fermer la connexion
    conn.close()

# Exécuter la fonction pour afficher le contenu des tables
show_tables()